# SerpAPI test

The AI overview block appears to be inside serpapi. Only problem is 100 query limit which hinders large scale generation

In [ ]:
# basic_google_search.py
from serpapi import GoogleSearch
import json

# Replace YOUR_API_KEY with your actual SerpApi key
params = {
    "engine": "google",
    "q": "how many birds in 12 days of christmas?",  # your search query
    "api_key": "7bd559239610628183d17af9371132f8349b0609027d8a401f1cd80993814438",
    "hl": "en",  # language
    "gl": "us"   # country
}

search = GoogleSearch(params)
results = search.get_dict()

# Print the entire JSON response
print("Full JSON results (pretty-printed):")
print(json.dumps(results, indent=2))

# If you want just the organic results:
organic = results.get("organic_results", [])
for idx, result in enumerate(organic, start=1):
    print(f"\nResult #{idx}")
    print("Title:", result.get("title"))
    print("Link:", result.get("link"))
    print("Snippet:", result.get("snippet"))



# Test with playwright

In [4]:
import asyncio
import json
from playwright.async_api import async_playwright
from playwright_stealth import stealth_async

async def scrape_google(query: str):
    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=False,  # Try headful to look more "real"
        )
        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                       "(KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"
        )
        page = await context.new_page()
        await stealth_async(page)  # Activate stealth

        print("Navigating to google.com ...")
        await page.goto("https://www.google.com/", timeout=30000)

        # Maybe accept cookies if needed
        # ...

        # "Human-like" delays
        await page.click("textarea[name='q']")
        await page.keyboard.type(query, delay=100)  # 100ms between keypress
        await page.keyboard.press("Enter")

        # Wait for results or fallback
        try:
            await page.wait_for_selector("div.g", timeout=15000)
        except:
            # Possibly got captcha page
            print("Likely got captcha/blocked, returning empty.")
            return []

        result_divs = await page.query_selector_all("div.g")
        results_data = []
        for idx, div in enumerate(result_divs, start=1):
            title_el = await div.query_selector("h3")
            if not title_el:
                continue
            title_text = await title_el.inner_text()
            link_el = await div.query_selector("a")
            link_href = await link_el.get_attribute("href") if link_el else None
            snippet_el = await div.query_selector("div[data-sncf]")
            snippet_text = await snippet_el.inner_text() if snippet_el else ""
            results_data.append({
                "position": idx,
                "title": title_text,
                "link": link_href,
                "snippet": snippet_text
            })

        await browser.close()
        return results_data

async def main():
    query_str = "how many birds in 12 days of christmas?"
    data = await scrape_google(query_str)
    print(json.dumps(data, indent=2))

await main()


TargetClosedError: BrowserType.launch: Target page, context or browser has been closed
Browser logs:

╔════════════════════════════════════════════════════════════════════════════════════════════════╗
║ Looks like you launched a headed browser without having a XServer running.                     ║
║ Set either 'headless: true' or use 'xvfb-run <your-playwright-app>' before running Playwright. ║
║                                                                                                ║
║ <3 Playwright Team                                                                             ║
╚════════════════════════════════════════════════════════════════════════════════════════════════╝
Call log:
  - <launching> /home/adanato/.cache/ms-playwright/chromium-1155/chrome-linux/chrome --disable-field-trial-config --disable-background-networking --disable-background-timer-throttling --disable-backgrounding-occluded-windows --disable-back-forward-cache --disable-breakpad --disable-client-side